# Download and process Global Land Cover Land Use 2019 data into a single geotiff at native resolution

In [2]:
import os
import sys
import shutil
import tarfile
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling                                                                                        

In [3]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [4]:
# Get the required info from the config file
data_path            = cs.read_from_config(config_file,'data_path')
geospatial_temp_path = cs.read_from_config(config_file,'geospatial_temp_path')
land_path            = cs.read_from_config(config_file,'land_path')
land_map_url         = cs.read_from_config(config_file,'land_map_url')
land_strata_url      = cs.read_from_config(config_file,'land_strata_url')
strata_legend        = cs.read_from_config(config_file,'land_legend')
download_area        = cs.read_from_config(config_file,'geospatial_area')

### Convert geospatial coordinates to specific download coordinates for this data product

In [8]:
download_coordinates = cs.geospatial_coordinates_to_download_coordinates(download_area, 'glclu2019')

Returning coordinates as type <class 'str'> for use with glclu2019 download code.


### Download data

In [45]:
glclu_lon,glclu_lat = cs.convert_coordinates_to_glclu2019_download_lists(download_coordinates)

In [58]:
download_folder_map    = Path(data_path) / geospatial_temp_path / 'glclu2019' / 'download' / 'map'
download_folder_strata = Path(data_path) / geospatial_temp_path / 'glclu2019' / 'download' / 'strata'

In [59]:
download_folder_map.mkdir(parents=True, exist_ok=True)
download_folder_strata.mkdir(parents=True, exist_ok=True)

In [60]:
for dl_lon in glclu_lon:
    for dl_lat in glclu_lat:
        map_url = land_map_url.format(dl_lat,dl_lon) # Replace placeholder values in url with download coordinates
        cs.download_glclu2019_grid(map_url,download_folder_map)
        
        strata_url = land_strata_url.format(dl_lat,dl_lon)
        cs.download_glclu2019_grid(strata_url,download_folder_strata)

Successfully downloaded https://storage.googleapis.com/earthenginepartners-hansen/GLCLU_2019/strata/60N_180W.tif
Successfully downloaded https://storage.googleapis.com/earthenginepartners-hansen/GLCLU_2019/map/60N_180W.tif
Successfully downloaded https://storage.googleapis.com/earthenginepartners-hansen/GLCLU_2019/strata/70N_180W.tif
Successfully downloaded https://storage.googleapis.com/earthenginepartners-hansen/GLCLU_2019/map/70N_180W.tif
Successfully downloaded https://storage.googleapis.com/earthenginepartners-hansen/GLCLU_2019/strata/30N_170W.tif
Successfully downloaded https://storage.googleapis.com/earthenginepartners-hansen/GLCLU_2019/map/30N_170W.tif
Successfully downloaded https://storage.googleapis.com/earthenginepartners-hansen/GLCLU_2019/strata/60N_170W.tif
Successfully downloaded https://storage.googleapis.com/earthenginepartners-hansen/GLCLU_2019/map/60N_170W.tif
Successfully downloaded https://storage.googleapis.com/earthenginepartners-hansen/GLCLU_2019/strata/70N_170W

### Merge

In [5]:
# Define the output file
merged_map_file = 'glclu2019_map.tif'
merged_strata_file = 'glclu2019_strata.tif'
merged_folder = Path(data_path) / geospatial_temp_path / 'glclu2019' / 'raw' # We'll end up with two files in here only, so no need to separate into 'map' and 'strata'

In [66]:
cs.merge_glclu2019_files_into_one(merged_map_file, download_folder_map, merged_folder, download_area)

In [68]:
cs.merge_glclu2019_files_into_one(merged_strata_file, download_folder_strata, merged_folder, download_area)

In [6]:
# Ensure we have the legend with the data
cs.download_url_into_folder(strata_legend, merged_folder)

Successfully downloaded https://glad.umd.edu/sites/default/files/legend_0.xlsx


### Delete the individual GeoTIFF files to save space

In [67]:
if os.path.isdir(download_folder_map): 
    shutil.rmtree(download_folder_map)

In [69]:
if os.path.isdir(download_folder_strata): 
    shutil.rmtree(download_folder_strata)

In [70]:
if os.path.isdir(download_folder): 
    shutil.rmtree(download_folder) # main folder that contained the 'map' and 'strata' folders